In [177]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv('data-logistic.csv', names=['y', 'x1', 'x2'])

In [109]:
x = df.drop('y', axis=1)
y = df.drop(['x1', 'x2'], axis=1)

In [159]:
def deriv_func(x, y, w, col):
    denom = 1 + np.e ** (-y * x.dot(w))
    return np.sum(x[col] * y * (1 - 1.0 / denom)) / x.size

In [160]:
deriv_func(x, np.ravel(y), np.array([0, 0]), 'x1')

0.17865633199370543

In [174]:
def gradient_desc(x, y, w, func=deriv_func ,step=0.1, 
                  max_iters=10000, eps=1e-5, l2=False, c=10):
    it = 0
    cur = w.copy()
    while it < max_iters:
        it += 1
        cur[0] += step * func(x, y, w, x.columns[0])
        cur[1] += step * func(x, y, w, x.columns[1])
        if l2:
            cur[0] -= step * c * w[0]
            cur[1] -= step * c * w[1]
        if np.linalg.norm(cur - w) < eps:
            w = cur
            return w
        w = cur.copy()
        

In [175]:
w = np.array([0.0, 0.0])
ws = gradient_desc(x, np.ravel(y), w)
wl2 = gradient_desc(x, np.ravel(y), w, l2=True)

In [176]:
ws, wl2

(array([ 0.28750254,  0.09226915]), array([ 0.01583043,  0.01404041]))

In [178]:
s = ws * x
axs = 1.0 / (1 + np.e ** (-s.x1 - s.x2))

In [181]:
s = wl2 * x
axl2 = 1.0 / (1 + np.e ** (-s.x1 - s.x2))

In [183]:
s_score = roc_auc_score(np.ravel(y), axs)

In [185]:
l2_score = roc_auc_score(np.ravel(y), axl2)

In [186]:
s_score, l2_score

(0.92685714285714282, 0.93666666666666654)